In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr  5 05:41:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install git+https://github.com/neuml/txtai datasets pandas
!pip install farasapy

Cloning into 'quranqa'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 261 (delta 35), reused 70 (delta 22), pack-reused 163
Receiving objects: 100% (261/261), 300.16 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (89/89), done.
  Cloning https://github.com/neuml/txtai to /tmp/pip-req-build-5fi9v9bj
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-req-build-5fi9v9bj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 325 kB 15.7 MB/s 
     |████████████████████████████████| 596 kB 77.9 MB/s 
     |████████████████████████████████| 6.5 MB 84.8 MB/s 
     |████████████████████████████████| 3.8 MB 54.1 MB/s 
     |████████████████████████████████| 8.6 MB 91.0 MB/s 
     |████████████████████████████████| 895 kB 84.9 MB/s 
     |████████████████████████

In [4]:
!pip install jsonlines
import jsonlines


import sys
sys.path.insert(1, '/content/quranqa/code')
import quranqa22_eval
from collections import defaultdict
from farasa.diacratizer import FarasaDiacritizer 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.pos import FarasaPOSTagger 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer
from gensim import corpora, models, similarities
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import EarlyStoppingCallback
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor
from txtai.pipeline import HFTrainer
from txtai.pipeline import Questions
from txtai.pipeline import Similarity
import operator
import pandas as pd
import pprint
import read_write_qrcd as q_reader
import torch, string, re, os, json

In [5]:
train_context_question_objects = q_reader.load_jsonl('./quranqa/datasets/qrcd_v1.1_train.jsonl')
dev_context_question_objects = q_reader.load_jsonl('./quranqa/datasets/qrcd_v1.1_dev.jsonl')
test_context_question_objects = q_reader.load_jsonl('./quranqa/datasets/qrcd_v1.1_test_noAnswers.jsonl')



Loaded 710 records from ./quranqa/datasets/qrcd_v1.1_train.jsonl
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
Loaded 274 records from ./quranqa/datasets/qrcd_v1.1_test_noAnswers.jsonl


In [6]:

train_data = [dict({"pq_id": context_question_object['pq_id'],
                    #"question": quranqa22_eval.normalize_text(context_question_object.question),
                    "question": context_question_object['question'],
                    "context": context_question_object['passage'], 
                    "answers": r['text']})
              for context_question_object in train_context_question_objects
              for r in context_question_object['answers']]

dev_data = [dict({"pq_id": context_question_object['pq_id'],
                  #"question": quranqa22_eval.normalize_text(context_question_object.question),
                  "question": context_question_object['question'],
                  "context": context_question_object['passage'],
                  "answers": r['text']})
            for context_question_object in dev_context_question_objects
            for r in context_question_object['answers']]

test_data = [dict({"pq_id": context_question_object['pq_id'],
                  #"question": quranqa22_eval.normalize_text(context_question_object.question),
                  "question": context_question_object['question'],
                  "context": context_question_object['passage'],
                   })
            for context_question_object in test_context_question_objects]

dev_train_data = train_data + dev_data

train_data2 = pd.read_csv('/content/drive/MyDrive/train_dataset.csv',delimiter='\t')
train_data2 = train_data2.to_dict('records')

dev_train_data2 = pd.read_csv('/content/drive/MyDrive/dev_train_dataset.csv',delimiter='\t')
dev_train_data2 = dev_train_data2.to_dict('records')

test_data2 = pd.read_csv('/content/drive/MyDrive/test_dataset.csv',delimiter='\t')
test_data2 = test_data2.to_dict('records')

'''with jsonlines.open('/content/drive/MyDrive/test_data2.jsonl', mode='w') as writer:
  for w in test_data2:
    writer.write(w)'''

"with jsonlines.open('/content/drive/MyDrive/test_data2.jsonl', mode='w') as writer:\n  for w in test_data2:\n    writer.write(w)"

In [7]:
test_data3 = []
with jsonlines.open('/content/drive/MyDrive/test_data2.jsonl', mode='w') as writer:
  for i,w in enumerate(test_data2):
    d0 = {'start_char': w['context'].find(w['answers'].strip()), 'text': w['answers'].strip()}
    d1={'pq_id':w['pq_id'],'question':w['question'],'passage':w['context'],'answers':[d0]}
    d2={'pq_id':w['pq_id'],'question':w['question'],'context':w['context'],'answers':[d0]}
    writer.write(d1)
    test_data3.append(d2)


In [8]:
for x in test_data2:
  if x['answers'] not in x['context']:
    print(x)

In [ ]:
dev_data_df = pd.DataFrame(dev_data)
dev_data_df.to_csv('dev.csv', sep='\t')
train_data_df = pd.DataFrame(train_data)
train_data_df.to_csv('train.csv', sep='\t')
test_data_df = pd.DataFrame(test_data)
test_data_df.to_csv('test.csv', sep='\t')

!cp *.csv /content/drive/MyDrive

In [ ]:
pretrained_models = [#"wissamantoun/araelectra-base-artydiqa", #num_train_epochs=2
                     #"salti/AraElectra-base-finetuned-ARCD", #num_train_epochs=8
                     "gfdgdfgdg/arap_qa_bert_large_v2",
                     #"gfdgdfgdg/arap_qa_bert_v2",
                     #"gfdgdfgdg/arap_qa_bert"
                     ]


def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['context'], 
                                 return_tensors='pt', padding=True, 
                                 truncation=True,max_length=512, 
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer,top_rank):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)
    
    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices
    
    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank > top_rank:
            break
        #answer = str(answer_start_v.item())+' '+ answer +' '+ str(answer_end_v.item())
    return answers


In [ ]:
def normalize_text(s):
    """remove punctuation, some stopwords and extra whitespace."""
    def remove_stopWords(text):
        terms = []
        stopWords = {'من', 'الى', 'إلى', 'عن', 'على', 'في', 'حتى'}
        for term in text.split():
            if term not in stopWords:
                terms.append(term)
        return " ".join(terms)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        # Arabic punctuation
        exclude.add('،')
        exclude.add('؛')
        exclude.add('؟')
        print(text)
        return ''.join(ch for ch in text if ch not in exclude)

    return white_space_fix(remove_stopWords((s)))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(pred, labels):
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}


In [ ]:

def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()

    return trainer(model_name, traindata,#devdata,
                   task="question-answering",
                   metric_for_best_model = 'f1',
                   #evaluation_strategy ='epoch',
                   #eval_steps = 500, # Evaluation and Save happens every 50 steps
                   #save_total_limit = 15,
                   save_strategy='no',
                   learning_rate=2e-5,
                   per_device_train_batch_size=batch_size,
                   #per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   #load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   )
    #return trainer(model_name, data, task="question-answering", num_train_epochs=num_train_epochs)#num_train_epochs=10

In [ ]:
def ir(data_item={}, query_key='question', docs_key='context', index_method='lsi', dims=4) -> list:
    # get data items 
    docs = [ver.strip() for ver in data_item.get(docs_key).split('.') if ver.strip()]
    query = data_item.get(query_key)
    stoplist = ''.split()
    texts = [
        [word for word in doc.split() if word not in stoplist]
        for doc in docs
    ]

    # build dictionary and corpus 
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # build tfidf & lsi models 
    tfidf = models.TfidfModel(corpus) 
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=dims)
    corpus_lsi = lsi_model[corpus_tfidf]

    # generate query BOW vector and LSI vector from the question 
    query_bow = query.replace('؟', '').replace('"', '').split()
    vec_bow = dictionary.doc2bow(query_bow)
    vec_lsi = lsi_model[vec_bow]

    result = []
    try:
        # generate tfidf and lsi indices 
        lsi_index = similarities.MatrixSimilarity(corpus_lsi)
        tfidf_index = similarities.MatrixSimilarity(corpus_tfidf)
        
        
        # get similarities between queries and docs 
        tfidf_sims = tfidf_index[vec_bow]
        lsi_sims = lsi_index[vec_lsi]

        # sort documents according to similarties 
        
        if index_method == 'tfidf':
            sims = tfidf_sims
        else:
            sims = lsi_sims
        
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        for doc_position, doc_score in sims:
            result.append( (doc_score, docs[doc_position]) )
    except:
        for doc in docs:
            result.append( (0.0, doc) )
    return result

In [ ]:
def filter_context(in_data):
  in_data = [dict(in_t) for in_t in in_data]

  out_data = []
  for data in in_data:
    filtered = ir(data_item=data)
    j=0
    for i in reversed(filtered): 
      j=j+1
      sc,worest = i
      context = data['context']
      if sc != 0:
        context = context.replace(worest,'')
      if j== 1:
        break
    context = context.replace('  ',' ')
    context = context.replace(' .','.')
    context = context.replace('..','.')
    data['context'] = context
    out_data.append(data)
  return out_data

In [ ]:

f_dev_data = filter_context(dev_data)
if f_dev_data == dev_data:
    print('0000000000000')


2022-03-30 10:14:00,739 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,743 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,751 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,755 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,763 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,767 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10:14:00,778 [WARNING] __init__: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-30 10

In [ ]:

def test_model2(tokenizer,model,data):
  questions = Questions(path=(model, tokenizer), gpu=True)
  result ={}
  for row in data:
    da = row['context'].split('.')
    qu = [row['question']]
    ans = list(set(questions([qu[0]] * len(da), da)))
    result[row["pq_id"]] = ans
  return result

def test_model(tokenizer,model,data,top_rank):
  result = {}
  for row in data:
    pred = get_prediction(row, model, tokenizer,top_rank)
    result[row["pq_id"]] = pred
  return result

In [ ]:
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run01.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run02.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run03.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run04.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run05.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/QQATeam_Run06.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'


[2022-03-30 10:15:57,496 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 274 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.5255006666996809, "exact_match": 0.2116788321167883, "f1": 0.46615448675765475}
[2022-03-30 10:16:06,687 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 274 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.5625351340918143, "exact_match": 0.24452554744525548, "f1": 0.5184042367570512}
[2022-03-30 10:16:15,746 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 274 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.5416605570150471, "exact_ma

In [ ]:
num_train_epochs=2
model_name = "wissamantoun/araelectra-base-artydiqa" #num_train_epochs=2
batch_size=1

print("\n\n###################"+model_name)
tokenizer_wissam = AutoTokenizer.from_pretrained(model_name)
model_wissam = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_wissam,model_wissam,test_data3,5)
open('./quranqa/datasets/wissam_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_RunID.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'

for num_train_epochs in range(2,20):
  model, tokenizer = Fine_tune(model_name,dev_train_data,test_data3,num_train_epochs,batch_size)
  #os.makedirs('wissam')
  model.save_pretrained('wissam')
  tokenizer.save_pretrained('wissam')
  tokenizer1 = AutoTokenizer.from_pretrained('./wissam')
  model1 = AutoModelForQuestionAnswering.from_pretrained('./wissam')
  result = test_model(tokenizer1,model1,test_data3,5)
  open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
  print(num_train_epochs,num_train_epochs,num_train_epochs)
  !python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'





###################wissamantoun/araelectra-base-artydiqa
[2022-03-30 06:58:08,614 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 1 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.14, "exact_match": 0.0, "f1": 0.14}


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss
500,2.483400
1000,2.162100
1500,1.280700


2 2 2
[2022-03-30 07:01:42,990 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 1 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.17391304347826086, "exact_match": 0.0, "f1": 0.17391304347826086}


Step,Training Loss
500,2.491000
1000,2.155900
1500,1.360900
2000,1.370500
2500,0.906000


...

13 13 13
[2022-03-30 09:31:47,142 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 1 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.14285714285714285, "exact_match": 0.0, "f1": 0.14285714285714285}


Step,Training Loss
500,2.502300
1000,2.174700
1500,1.444300
2000,1.504600
2500,1.021600
3000,1.294600
3500,0.977100
4000,0.943700
4500,0.767300
5000,0.789600


14 14 14
[2022-03-30 09:55:32,334 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 1 records from /content/drive/MyDrive/test_data2.jsonl
The run file is correct.
{"pRR": 0.30434782608695654, "exact_match": 0.0, "f1": 0.30434782608695654}


Step,Training Loss
500,2.502300
1000,2.182900
1500,1.484800
2000,1.500200
2500,1.087800
3000,1.257900
3500,0.959000
4000,0.944200
4500,0.752600
5000,0.840500


KeyboardInterrupt: ignored

In [ ]:
model_name = "salti/AraElectra-base-finetuned-ARCD" #num_train_epochs=2
num_train_epochs=8
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,test_data3,5)
open('./quranqa/datasets/salti_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_RunID.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'

model_salti2, tokenizer_salti2 = Fine_tune(model_name,dev_train_data,test_data3,num_train_epochs,batch_size)
os.makedirs('salti')
model_salti2.save_pretrained('salti')
tokenizer_salti2.save_pretrained('salti')
tokenizer_salti2 = AutoTokenizer.from_pretrained('./salti')
model_salti2 = AutoModelForQuestionAnswering.from_pretrained('./salti')
result = test_model(tokenizer_salti2,model_salti2,test_data3,5)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file '/content/drive/MyDrive/test_data2.jsonl'



In [ ]:
model_name = "salti/AraElectra-base-finetuned-ARCD" #num_train_epochs=2
num_train_epochs=8
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data,5)
open('./quranqa/datasets/salti_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
os.makedirs('salti')
model.save_pretrained('salti')
tokenizer.save_pretrained('salti')
tokenizer = AutoTokenizer.from_pretrained('./salti')
model = AutoModelForQuestionAnswering.from_pretrained('./salti')
result = test_model(tokenizer,model,dev_data,5)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'
'''

In [ ]:
def post_ir(query, docs, index_method='lsi', dims=4):
    texts = [
        [word for word in doc.split()]
        for doc in docs
    ]

    # build dictionary and corpus 
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # build tfidf & lsi models 
    tfidf = models.TfidfModel(corpus) 
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=dims)
    corpus_lsi = lsi_model[corpus_tfidf]

    # generate query BOW vector and LSI vector from the question 
    query_bow = query.replace('؟', '').replace('"', '').split()
    vec_bow = dictionary.doc2bow(query_bow)
    vec_lsi = lsi_model[vec_bow]

    result = []
    try:
        # generate tfidf and lsi indices 
        lsi_index = similarities.MatrixSimilarity(corpus_lsi)
        tfidf_index = similarities.MatrixSimilarity(corpus_tfidf)
        
        
        # get similarities between queries and docs 
        tfidf_sims = tfidf_index[vec_bow]
        lsi_sims = lsi_index[vec_lsi]

        # sort documents according to similarties 
        
        if index_method == 'tfidf':
            sims = tfidf_sims
        else:
            sims = lsi_sims
        
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        for doc_position, doc_score in sims:
            result.append( (doc_score, docs[doc_position]) )
    except:
        for doc in docs:
            result.append( (0.0, doc) )
    return result

In [ ]:
def rescoring(dev_data,result):
  results1 = {}
  results2 = {}
  results3 = {}
  results4 = {}
  scores = []
  for row in dev_data:
    for ans in result[row['pq_id']]:
      scores.append(ans['score'])

  for row in dev_data:
    rank = 1
    l = []
    l2 = []
    
    for ans in result[row['pq_id']]:
      answer = ans['answer']
      score = ans['score']
      scores_norm = (score-min(scores))/(max(scores)-min(scores))
      if '؟' not in answer:
        ans = answer.split('.')
        for an in ans:
          if an!='' and '#' not in an:
            r = {'answer':an,'rank':rank,'score':scores_norm}
            if rank<6:
              l.append(r)
            l2.append(r)
            rank = rank +1
            scores_norm = scores_norm - 0.0001
            
    results1[row['pq_id']] = l 
    results3[row['pq_id']] = l2

    scores0 = {}
    scores1 = {}
    scores2 = {}

    docs = []
    query = row['question']
    for ans in results3[row['pq_id']]:
      docs.append(ans['answer'])
      scores1[ans['answer']] = ans['score']

    sim_scores = post_ir(query, docs)
    for sc, snt in sim_scores:
      scores2[snt] = sc

    for k in scores1.keys():
      scores0[k] = scores1[k]+(scores2[k]+0.01)

    scores0 = dict(sorted(scores0.items(), key=lambda item: item[1], reverse=True))


    l = []
    rank = 1
    for k in scores0.keys():
      r = {'answer':k,'rank':rank,'score':scores0[k]}
      if rank<6:
        l.append(r)
      l2.append(r)
      rank = rank +1

    results2[row['pq_id']] = l 
    results4[row['pq_id']] = l2

  return results1,results2,results3,results4


In [ ]:
num_train_epochs=0

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/wissamantoun')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/wissamantoun')

result = test_model(tokenizer,model,dev_data,5)
result2 = test_model2(tokenizer,model,dev_data)
open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/wissamantoun')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/wissamantoun')

result = test_model(tokenizer,model,dev_data,20)


In [ ]:
results11,results12,results13,results14 = rescoring(dev_data,result)

In [ ]:
num_train_epochs=1
open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(results11,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

num_train_epochs=2
open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(results12,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


[2022-03-27 20:27:46,984 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6442512889184745, "exact_match": 0.3669724770642202, "f1": 0.5952395968816058}
[2022-03-27 20:27:53,859 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6124976858881551, "exact_match": 0.3486238532110092, "f1": 0.5560022602483338}


In [ ]:
num_train_epochs=0
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/salti')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/salti')

result = test_model(tokenizer,model,dev_data,5)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


[2022-03-27 20:28:54,743 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6243208403005245, "exact_match": 0.3669724770642202, "f1": 0.5867165193384595}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/salti')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/salti')

result = test_model(tokenizer,model,dev_data,20)


In [ ]:
results21,results22,results23,results24 = rescoring(dev_data,result)

In [ ]:
num_train_epochs=1
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(results21,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

num_train_epochs=2
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(results22,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


[2022-03-27 20:29:41,221 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.609003748199461, "exact_match": 0.3486238532110092, "f1": 0.5549594982285045}
[2022-03-27 20:29:47,893 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.5593736585071185, "exact_match": 0.3211009174311927, "f1": 0.5004948069154099}


In [ ]:
with open('./quranqa/datasets/wissam_Run1F.json', 'r') as f:
  wissam_RunIDF = json.load(f)

with open('./quranqa/datasets/salti_Run0F.json', 'r') as f:
  salti_RunIDF = json.load(f)


In [ ]:

result = {}
for wissam_pq_dict,_ in zip(wissam_RunIDF,salti_RunIDF):
    ans_score = dict()
    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]*1
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]*1
        
    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]*0.1
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]*0.1
     
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/w2s11_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/w2s11_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-27 20:30:01,793 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6176734894102516, "exact_match": 0.3761467889908257, "f1": 0.5741020757810674}


In [ ]:

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings({"path": model_name,"context":3})

# Create extractor instance
extractor = Extractor(embeddings, model_name)

Some weights of the model checkpoint at salti/AraElectra-base-finetuned-ARCD were not used when initializing ElectraModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

# Create and run pipeline
similarity = Similarity()
q = "feel good story"
d = ["Maine man wins $1M from $25 lottery ticket", 
    "Don't sacrifice slower friends in a bear attack"]
similarity(q, d)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


[(0, 0.6845324039459229), (1, 0.0025504184886813164)]

In [ ]:

cont = 'وإذ ابتلى إبراهيم ربه بكلمات فأتمهن قال إني جاعلك للناس إماما قال ومن ذريتي قال لا ينال عهدي الظالمين. وإذ جعلنا البيت مثابة للناس وأمنا واتخذوا من مقام إبراهيم مصلى وعهدنا إلى إبراهيم وإسماعيل أن طهرا بيتي للطائفين والعاكفين والركع السجود. وإذ قال إبراهيم رب اجعل هذا بلدا آمنا وارزق أهله من الثمرات من آمن منهم بالله واليوم الآخر قال ومن كفر فأمتعه قليلا ثم أضطره إلى عذاب النار وبئس المصير. وإذ يرفع إبراهيم القواعد من البيت وإسماعيل ربنا تقبل منا إنك أنت السميع العليم. ربنا واجعلنا مسلمين لك ومن ذريتنا أمة مسلمة لك وأرنا مناسكنا وتب علينا إنك أنت التواب الرحيم. ربنا وابعث فيهم رسولا منهم يتلو عليهم آياتك ويعلمهم الكتاب والحكمة ويزكيهم إنك أنت العزيز الحكيم.'
q = 'من بنى الكعبة'
qu = ['من بنى الكعبة']
data = cont.split('.')

# Create and run pipeline
similarity = Similarity('./wissam')
similarity(q, data)

OSError: ignored

In [ ]:

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2"})

# Create extractor instance
extractor = Extractor(embeddings, "distilbert-base-cased-distilled-squad")
data = ["Giants hit 3 HRs to down Dodgers",
        "Giants 5 Dodgers 4 final",
        "Dodgers drop Game 2 against the Giants, 5-4",
        "Blue Jays beat Red Sox final score 2-1",
        "Red Sox lost to the Blue Jays, 2-1",
        "Blue Jays at Red Sox is over. Score: 2-1",
        "Phillies win over the Braves, 5-0",
        "Phillies 5 Braves 0 final",
        "Final: Braves lose to the Phillies in the series opener, 5-0",
        "Lightning goaltender pulled, lose to Flyers 4-1",
        "Flyers 4 Lightning 1 final",
        "Flyers win 4-1"]

questions = ["What team won the game?", "What was score?"]

extractor([(question, question, question, False) for question in questions], data)


[('What team won the game?', 'Braves'), ('What was score?', '2-1')]

In [ ]:
dev_data[0]

{'answers': 'للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل',
 'context': 'إنما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضة من الله والله عليم حكيم. ومنهم الذين يؤذون النبي ويقولون هو أذن قل أذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمة للذين آمنوا منكم والذين يؤذون رسول الله لهم عذاب أليم.',
 'pq_id': '9:60-61_316',
 'question': 'ما هي مصارف الزكاة؟'}

In [ ]:

result ={}
tokenizer = AutoTokenizer.from_pretrained('./wissam')
model = AutoModelForQuestionAnswering.from_pretrained('./wissam')
questions = Questions(path=(model, tokenizer), gpu=True)
i=0
for row in dev_data:
  data = row['context'].split('.')
  qu = [row['question']]
  rank = 1
  
  answers = []

  ans = list(set(questions([qu[0]] * len(data), data)))
  for r in ans:
    if r != None:
      answer = {}
      answer["answer"] = r
      answer["rank"] = rank
      answer["score"] = 5 - rank
      rank = rank +1
      answers.append(answer)
    if rank == 6:
        break
        
  if len(answers):
    result[row["pq_id"]] = answers
  i = i+1
  '''if i==18:
    print(answers)
    break'''


open('./quranqa/datasets/Que_Run1F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/Que_Run1F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:999: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[2022-03-25 14:48:57,742 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
Unanswered question 9:122-123_400 will receive score 0.
Unanswered question 39:9-10_313 will receive score 0.
Unanswered question 33:56-59_348 will receive score 0.
Unanswered question 3:164-165_415 will receive score 0.
Unanswered question 48:16-17_415 will receive score 0.
Unanswered question 49:6-8_415 will receive score 0.
{"pRR": 0.32898166805660595, "exact_match": 0.1559633027522936, "f1": 0.28199300445676023}


In [ ]:

result['2:178-179_400']



[{'answer': 'ولكم في القصاص حياة', 'rank': 1, 'score': 4},
 {'answer': 'كتب عليكم القصاص في القتلى الحر بالحر والعبد بالعبد والأنثى بالأنثى',
  'rank': 2,
  'score': 3}]

In [9]:
!python ./quranqa/code/quranqa22_eval.py --run_file '/content/drive/MyDrive/results/paper/QQATeam_Run1.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
100% 241M/241M [00:13<00:00, 18.0MiB/s]
[2022-04-05 05:45:42,382 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6313448903022509, "exact_match": 0.3669724770642202, "f1": 0.6030726642357181}
